In [13]:
import pandas as pd
import logging

logger = logging.getLogger(__name__)
# Load the CSV file
file_path = 'full_cohort_data.csv'
df = pd.read_csv(file_path)

In [14]:
# Keep track of columns to be removed
removed_columns = []

# Identify and remove columns that have only 0 (zeros) in them
zero_columns = df.columns[(df == 0).all()]
logger.debug(f"Zero columns: {zero_columns}")
removed_columns.extend(zero_columns)
df = df.loc[:, (df != 0).any(axis=0)]

# Identify and remove columns that have non-numeric values
non_numeric_columns = df.select_dtypes(exclude=[int, float]).columns
removed_columns.extend(non_numeric_columns)
df = df.select_dtypes(include=[int, float])

# Identify and remove columns that have more than 100 missing values
missing_value_columns = df.columns[df.isnull().sum() > 100]
removed_columns.extend(missing_value_columns)
df = df.dropna(thresh=df.shape[0] - 100, axis=1)

# Save the cleaned DataFrame back to the CSV file
df.to_csv(file_path, index=False)

i removed the columns that had only 0 in it.
i removed the columns that had a non-numerical values.
i removed the columns that had more than 100 missing values.
the columns that i removed so far are: 'weight_first', 'bmi', 'service_unit', 'day_icu_intime', 'sepsis_flg', 'po2_first', 'pco2_first, 'days_28_flg'.   

In [ ]:
relevant = ["", "hosp_exp_flg", "icu_exp_flg", "mort_day_censored", "censor_flg" ,"hour_icu_intime", 'map_1st' , "hospital_los_day",  "icu_los_day" ]
print(df.columns)
for column in relevant:
    if column in df.columns:
        removed_columns.append(column)
        df = df.drop(columns=[column])
   
        print(f"Removed column: {column}")
    else:
       print(f"Column not found: {column}")                

I removed columns that directly indicated survival outcomes. The purpose of the exercise is to predict human survival based on data. These columns give a sure prediction, so I removed them. 
the columns I removed here are: